In [ ]:
import findspark
findspark.init("/home/ubuntu/spark-2.1.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BDAS-Linda').getOrCreate()

from functools import reduce
from pyspark.sql import DataFrame


In [ ]:
# import datasets
filename = "./Subnational-ethnic-population-projections-2013base.csv"
filename1 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2004-2008.csv"
filename2 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2009-2013.csv"
filename3 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2014-2018.csv"
filename4 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2019.csv"

POP=spark.read.load(filename, format="csv", header="true")
MLLSN1=spark.read.load(filename1, format="csv", header="true")
MLLSN2=spark.read.load(filename2, format="csv", header="true")
MLLSN3=spark.read.load(filename3, format="csv", header="true")
MLLSN4=spark.read.load(filename4, format="csv", header="true")

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

MLLSN=unionAll(MLLSN1,MLLSN2,MLLSN3,MLLSN4)


In [ ]:

POP.count()

In [ ]:
MLLSN.count()